# **Municipality Correlations**

## **Datasets**

For this study we will require data from three different files:

- Population by municipality in 2023.
- Number of monthly tourists by municipality of destination in 2023.
- Total number of establishments dedicated to tourism in 2023.

We will combine the data from these three files into a single dataset to obtain a series of variables related to tourism in such a way that we can check the correlation between them.

Finally, since the data of all the Spanish tourist establishments (except those of the Canary Islands) are downloaded in a single file, it is necessary to know the geometry of the different municipalities that constitute the country in order to be able to locate each one of these accomodations.

## **Goal**

The aim of this study is to examine the correlation between the ratio of tourists/establishments and the population in some of the Spanish municipalities.


## **Useful Links:**

 - Population by municipality: https://www.ine.es/dynt3/inebase/index.htm?padre=525
 - Tourists by municipality: https://www.ine.es/dynt3/inebase/index.htm?padre=8578&capsel=8578 (Número de turistas mensuales por municipio de destino, desglosados por continente y país de residencia - Año 2023).
 - Geographical limits of Spanish municipalities: https://opendata.esri.es/datasets/ComunidadSIG::municipios-ign/explore?location=38.140848%2C-3.660650%2C5.95

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_dir_geofabrik = '/content/gdrive/MyDrive/TFM/Geofabrik/'
data_dir_geom = '/content/gdrive/MyDrive/TFM/Geometry/'
data_dir_INE = '/content/gdrive/MyDrive/TFM/INE/'

In [ ]:
!pip install osmium

In [ ]:
!pip install pandas fiona shapely pyproj rtree

In [ ]:
!pip install geopandas

In [ ]:
# Import necessary packages
import osmium as osm
import pandas as pd
import geopandas as gpd
import numpy as np
import itertools
from shapely.geometry import Point, LineString, Polygon
from shapely import wkb
from geopy.distance import distance

## 1. Load and Analyze Data

In a first step, we are going to download and analyze the datasets we have in order to better understand their structure and the information they contain. Given that Spain has a large number of municipalities, we will focus only on certain provinces. If necessary, we will analyze data for individual municipalities later on.

The first dataset we are going to read is the one that contains the data of the monthly number of tourists received in each municipality throughout 2023. Given that the Excel file has the data for each month in different sheets, we are going to use a *for* loop to convert each of them into a dataframe and then concatenate them all.

In [ ]:
# Read the monthly data of tourists received in each municipality during 2023
df_tourists = pd.read_excel(data_dir_INE + 'exp_tmov_receptor_mun_2023.xlsx', sheet_name='m01_2023')

for m in range(2, 13, 1):

    if m == 2:
      df_tourists = pd.concat([df_tourists, pd.read_excel(data_dir_INE + 'exp_tmov_receptor_mun_2023.xlsx', sheet_name='m0' + str(m) + '_2023')], axis = 0)

    elif 2 < m < 10:
      df_tourists = pd.concat([df_tourists, pd.read_excel(data_dir_INE + 'exp_tmov_receptor_mun_2023.xlsx', sheet_name='m0' + str(m) + '_2023')], axis = 0)

    else:
      df_tourists = pd.concat([df_tourists, pd.read_excel(data_dir_INE + 'exp_tmov_receptor_mun_2023.xlsx', sheet_name='m' + str(m) + '_2023')], axis = 0)

df_tourists

,mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,prov_dest_cod,prov_dest
0,2023-01,0,Total,1002,Amurrio,258,1,Araba/Álava
1,2023-01,10,Total Europa,1002,Amurrio,235,1,Araba/Álava
2,2023-01,11,Total Unión Europea,1002,Amurrio,214,1,Araba/Álava
3,2023-01,110,Francia,1002,Amurrio,58,1,Araba/Álava
4,2023-01,126,Alemania,1002,Amurrio,47,1,Araba/Álava
...,...,...,...,...,...,...,...,...
30271,2023-12,125,Reino Unido,52001,Melilla,40,52,Melilla
30272,2023-12,126,Alemania,52001,Melilla,296,52,Melilla
30273,2023-12,131,Suecia,52001,Melilla,190,52,Melilla
30274,2023-12,203,Argelia,52001,Melilla,32,52,Melilla


In [ ]:
df_tourists.dtypes

mes              object
pais_orig_cod     int64
pais_orig        object
mun_dest_cod      int64
mun_dest         object
turistas          int64
prov_dest_cod     int64
prov_dest        object
dtype: object

In [ ]:
df_tourists['prov_dest'].unique()

array(['Araba/Álava', 'Albacete', 'Alicante/Alacant', 'Almería', 'Ávila',
       'Badajoz', 'Balears, Illes', 'Barcelona', 'Burgos', 'Cáceres',
       'Cádiz', 'Castellón/Castelló', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Girona', 'Granada', 'Guadalajara',
       'Gipuzkoa', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Rioja, La', 'Lugo', 'Madrid', 'Málaga', 'Murcia', 'Navarra',
       'Ourense', 'Asturias', 'Palencia', 'Palmas, Las', 'Pontevedra',
       'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria', 'Segovia',
       'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia/València', 'Valladolid', 'Bizkaia', 'Zamora', 'Zaragoza',
       'Ceuta', 'Melilla'], dtype=object)

As can be seen, the generated dataframe contains a large number of rows recording the number of tourists received between January and December 2023 differentiated by country of origin. With so much data it is very difficult to understand how exactly the number of tourists is reflected in each municipality, so we will initially focus on a single province and just on a few municipalities.

We will select **Melilla**, as it only consists of one municipality.

In [ ]:
df_Melilla = df_tourists[df_tourists['prov_dest'] == 'Melilla']
df_Melilla

,mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,prov_dest_cod,prov_dest
24049,2023-01,0,Total,52001,Melilla,12648,52,Melilla
24050,2023-01,10,Total Europa,52001,Melilla,1904,52,Melilla
24051,2023-01,11,Total Unión Europea,52001,Melilla,1773,52,Melilla
24052,2023-01,20,Total África,52001,Melilla,10547,52,Melilla
24053,2023-01,30,Total América,52001,Melilla,101,52,Melilla
...,...,...,...,...,...,...,...,...
30271,2023-12,125,Reino Unido,52001,Melilla,40,52,Melilla
30272,2023-12,126,Alemania,52001,Melilla,296,52,Melilla
30273,2023-12,131,Suecia,52001,Melilla,190,52,Melilla
30274,2023-12,203,Argelia,52001,Melilla,32,52,Melilla


As can be seen, the number of rows has been considerably reduced.

First of all, let's check that there is indeed only data for the municipality of Melilla.

In [ ]:
df_Melilla['mun_dest'].unique()

array(['Melilla'], dtype=object)

Once this is confirmed, and given that what we are interested in is to have all the tourist data grouped by month regardless of the country of origin of the tourists, we will check the unique values of the *pais_orig* column.

In [ ]:
df_Melilla['pais_orig'].unique()

array(['Total', 'Total Europa', 'Total Unión Europea', 'Total África',
       'Total América', 'Total América del Norte', 'Total Asia',
       'Bélgica', 'Francia', 'Italia', 'Países Bajos', 'Polonia',
       'Reino Unido', 'Alemania', 'Suecia', 'Marruecos', 'China',
       'Total Sudamérica', 'Suiza', 'Filipinas', 'EE.UU.', 'Argelia',
       'Irlanda', 'Austria', 'Dinamarca', 'Noruega', 'Luxemburgo',
       'Total Centroamérica y Caribe'], dtype=object)

We can see that one of the options is *Total*, so it would make sense that only with this record we already have the total number of tourists received in each municipality each month. To make this easier to check, let's see the values for a single month.

In [ ]:
df_Melilla_m = df_Melilla[df_Melilla['mes'] == '2023-01']
df_Melilla_m

,mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,prov_dest_cod,prov_dest
24049,2023-01,0,Total,52001,Melilla,12648,52,Melilla
24050,2023-01,10,Total Europa,52001,Melilla,1904,52,Melilla
24051,2023-01,11,Total Unión Europea,52001,Melilla,1773,52,Melilla
24052,2023-01,20,Total África,52001,Melilla,10547,52,Melilla
24053,2023-01,30,Total América,52001,Melilla,101,52,Melilla
24054,2023-01,31,Total América del Norte,52001,Melilla,45,52,Melilla
24055,2023-01,40,Total Asia,52001,Melilla,96,52,Melilla
24056,2023-01,103,Bélgica,52001,Melilla,100,52,Melilla
24057,2023-01,110,Francia,52001,Melilla,461,52,Melilla
24058,2023-01,115,Italia,52001,Melilla,73,52,Melilla


After reviewing the different values, we see that there are tourists that are recorded up to 4 times. For example, tourists coming from Sweden are recorded in that country, in *Total Europa*, in *Total Unión Europea* and in *Total*. Therefore, we have checked whether aggregating the total numbers for each continent would result in the value that appears in Total. Indeed, we see that *Total Europa* (1904) + *Total África* (10547) + *Total América* (101) + *Total Asia* (96) = *Total* (12648). Therefore, we will be left with only that record to carry out our analyses.

Now, as we did in the *Tourism_Capacity_2023_EDA* notebook, we are going to read the data on tourism establishments registered in Spain during 2023. These data will then be located on a map of Spain in order to perform a count in the selected municipalities.

As explained previously, to be able to extract the data on tourist establishments from a .pbf file, we will use the **Pyosmium** library, which allows reading and extracting information from OSM files through the **SimpleHandler** class. Therefore, we will now define an instance of this class and the functions that will allow us to carry out this reading.

In [ ]:
class POIHandler(osm.SimpleHandler):
    '''
    Class to extract information from an osm.pbf file. Only elements identified
    as 'node' or 'area' are extracted. In addition, a filtering can be applied to
    select only those that have tags with a certain key and value.

    The position of the areas is obtained by calculating the centroid of the
    polygon formed by their nodes.

    Arguments
    ---------

    custom_filter: dict
        Dictionary with the keys and values that the elements must have to be
        extracted. For example:

        `{'amenity': ['restaurant', 'bar']}` selects only those elements that
        have the key 'amenity' with value 'restaurant' or 'bar'.

        `{'amenity': ['restaurant', 'bar'], 'building': ['car']}` selects only
        those elements that have the key 'amenity' with value 'restaurant' or
        'bar', or those with the key 'building' with value 'hotel'.
    '''

    def __init__(self, custom_filter=None):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.custom_filter = custom_filter

        if self.custom_filter:
            for key, value in self.custom_filter.items():
                if isinstance(value, str):
                    self.custom_filter[key] = [value]

    def node(self, node):
        if self.custom_filter is None:
            name = node.tags.get('name', '')
            self.tag_inventory(node, 'node', name)
        else:
            if any([node.tags.get(key) in self.custom_filter[key] for key in self.custom_filter.keys()]):
                name = node.tags.get('name', '')
                self.tag_inventory(node, 'node', name)

    def area(self, area):
        if self.custom_filter is None:
            name = area.tags.get('name', '')
            self.tag_inventory(area, 'area', name)
        else:
            if any([area.tags.get(key) in self.custom_filter[key] for key in self.custom_filter.keys()]):
                name = area.tags.get('name', '')
                self.tag_inventory(area, 'area', name)

    def tag_inventory(self, elem, elem_type, name):
        if elem_type == 'node':
            for tag in elem.tags:
                self.osm_data.append([elem_type,
                                       elem.id,
                                       name,
                                       elem.location.lon,
                                       elem.location.lat,
                                       pd.Timestamp(elem.timestamp),
                                       len(elem.tags),
                                       tag.k,
                                       tag.v])
        if elem_type == 'area':
            try:
                # A Polygon is created with the nodes that form the area to
                # calculate its centroid.
                nodes = list(elem.outer_rings())[0]
                polygon = Polygon([(node.lon, node.lat) for node in nodes])
                for tag in elem.tags:
                    self.osm_data.append([elem_type,
                                           elem.id,
                                           name,
                                           polygon.centroid.x,
                                           polygon.centroid.y,
                                           pd.Timestamp(elem.timestamp),
                                           len(elem.tags),
                                           tag.k,
                                           tag.v])
            except:
                pass

As we are not going to analyze the municipalities of the Canary Islands, we will only read the data from the file corresponding to the rest of Spain.

In [ ]:
# OSM markers Spain: extraction of elements identified with 'tourism'= ['dictionary with values of interest']
poi_handler_sp = POIHandler(custom_filter={'tourism':['alpine_hut', 'apartment', 'camp_pitch', 'camp_site', 'caravan_site', 'chalet',
                                          'guest_house', 'hostel', 'hotel', 'motel', 'wilderness_hut']})
poi_handler_sp.apply_file(data_dir_geofabrik + 'Spain/2023/spain-240101.osm.pbf')

As a result of the extraction, a list is obtained in which each element is in turn a list with the information associated with a node or area. To facilitate the management of the extracted data, we are going to store this information in a pandas dataframe.

In [ ]:
colnames = ['type', 'id', 'name', 'lon', 'lat', 'timestamp','n_tags', 'tag_key',
            'tag_value']
df_estab = pd.DataFrame(poi_handler_sp.osm_data, columns=colnames)
df_estab.head()

,type,id,name,lon,lat,timestamp,n_tags,tag_key,tag_value
0,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,addr:city,Madrid
1,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,addr:country,ES
2,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,addr:housenumber,15
3,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,addr:postcode,28012
4,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,addr:street,Calle del Calvario


Once this is done, we will be left with only one tag in order to avoid counting a node as many times as tags it has associated. Since, theoretically, the only tag associated with all the nodes is **tourism** (we will see below that the nodes have different numbers of tags), we will use this tag.

In [ ]:
df_filt_estab = df_estab[df_estab['tag_key'].isin(['tourism'])]
df_filt_estab

,type,id,name,lon,lat,timestamp,n_tags,tag_key,tag_value
6,node,21947483,,-3.702375,40.411544,2022-04-01 20:07:02+00:00,7,tourism,apartment
17,node,25913327,NH Ciudad de la Imagen,-3.788176,40.398441,2022-02-19 10:04:33+00:00,13,tourism,hotel
22,node,26860899,Buchaca,1.357838,42.373091,2019-10-05 08:59:10+00:00,3,tourism,camp_site
24,node,26860903,Cadaques,3.281944,42.288056,2011-09-11 16:11:28+00:00,2,tourism,hotel
27,node,26860947,El Quinto,-1.859175,37.140828,2007-12-08 01:04:36+00:00,3,tourism,camp_site
...,...,...,...,...,...,...,...,...,...
182681,area,2470638926,Área para Autocaravanas de Peralada,3.008696,42.305652,2023-12-30 00:16:51+00:00,10,tourism,caravan_site
182689,area,2470956308,Área de Belchite,-0.749649,41.306398,2023-12-30 10:53:13+00:00,9,tourism,caravan_site
182701,area,2470956314,Área de Andorra,-0.447151,40.983722,2023-12-30 10:53:13+00:00,10,tourism,caravan_site
182712,area,2471008518,Área de autocaravanas Peralta de la Sal,0.389054,41.993916,2023-12-30 13:47:33+00:00,11,tourism,caravan_site


Next, to ensure that we do not have duplicates, we will delete those records that may have the same name, except in the case where there is an empty value.

In [ ]:
# Eliminate establishments that may be duplicated by name except when it is an empty value

# Filter rows where 'name' is not empty and remove duplicates, keeping the first occurrence
name_no_empty = df_filt_estab[df_filt_estab['name'] != ''].drop_duplicates(subset='name', keep='first')

# Filter rows where 'name' is empty
name_empty = df_filt_estab[df_filt_estab['name'] == '']

# Concatenate the DataFrames
estab_cleaned = pd.concat([name_no_empty, name_empty])

estab_cleaned

,type,id,name,lon,lat,timestamp,n_tags,tag_key,tag_value
17,node,25913327,NH Ciudad de la Imagen,-3.788176,40.398441,2022-02-19 10:04:33+00:00,13,tourism,hotel
22,node,26860899,Buchaca,1.357838,42.373091,2019-10-05 08:59:10+00:00,3,tourism,camp_site
24,node,26860903,Cadaques,3.281944,42.288056,2011-09-11 16:11:28+00:00,2,tourism,hotel
27,node,26860947,El Quinto,-1.859175,37.140828,2007-12-08 01:04:36+00:00,3,tourism,camp_site
29,node,26860948,El Sur,-5.171844,36.721090,2012-09-27 16:10:10+00:00,2,tourism,hostel
...,...,...,...,...,...,...,...,...,...
182272,area,2446745740,,2.152927,41.387622,2023-11-14 04:57:33+00:00,3,tourism,apartment
182417,area,2453486016,,-1.116294,38.601567,2023-12-25 19:08:43+00:00,3,tourism,caravan_site
182459,area,2457900078,,-7.702241,42.694717,2023-12-09 13:36:42+00:00,2,tourism,hostel
182489,area,33714091,,-4.114507,36.725325,2023-12-21 21:54:06+00:00,2,tourism,caravan_site


After this filtering, one last step would be missing: to obtain the geometry column of each of the establishments from the longitude and latitude values.

In [ ]:
# Obtain geometry for each establishment
geometry = []

for i in range(len(estab_cleaned)):
    geometry.append(Point(estab_cleaned.iloc[i]['lon'], estab_cleaned.iloc[i]['lat']))

estab_cleaned = estab_cleaned.set_geometry(geometry)
estab_cleaned

,type,id,name,lon,lat,timestamp,n_tags,tag_key,tag_value,geometry
17,node,25913327,NH Ciudad de la Imagen,-3.788176,40.398441,2022-02-19 10:04:33+00:00,13,tourism,hotel,POINT (-3.78818 40.39844)
22,node,26860899,Buchaca,1.357838,42.373091,2019-10-05 08:59:10+00:00,3,tourism,camp_site,POINT (1.35784 42.37309)
24,node,26860903,Cadaques,3.281944,42.288056,2011-09-11 16:11:28+00:00,2,tourism,hotel,POINT (3.28194 42.28806)
27,node,26860947,El Quinto,-1.859175,37.140828,2007-12-08 01:04:36+00:00,3,tourism,camp_site,POINT (-1.85918 37.14083)
29,node,26860948,El Sur,-5.171844,36.721090,2012-09-27 16:10:10+00:00,2,tourism,hostel,POINT (-5.17184 36.72109)
...,...,...,...,...,...,...,...,...,...,...
182272,area,2446745740,,2.152927,41.387622,2023-11-14 04:57:33+00:00,3,tourism,apartment,POINT (2.15293 41.38762)
182417,area,2453486016,,-1.116294,38.601567,2023-12-25 19:08:43+00:00,3,tourism,caravan_site,POINT (-1.11629 38.60157)
182459,area,2457900078,,-7.702241,42.694717,2023-12-09 13:36:42+00:00,2,tourism,hostel,POINT (-7.70224 42.69472)
182489,area,33714091,,-4.114507,36.725325,2023-12-21 21:54:06+00:00,2,tourism,caravan_site,POINT (-4.11451 36.72533)


Now we will be able to count the number of establishments in the following section.

It is important to note that in this section we have not analyzed the structure of the dataset containing the population data of the municipalities of the selected provinces. We have preferred to leave this for the next section, as this will allow us to filter the geometries we are interested in from all those available in the municipal boundaries dataset. Therefore, we will now proceed to count the number of establishments in the municipalities of interest.

## 2. Count and Prepare Data

Now, as we did in previous studies, we will access the geometry of Spain, divided into municipalities, and we will count the different tourist points in each one of them. Finally, we will create a dataset for each of the provinces we consider interesting to analyze.

In [ ]:
# Access geometry of Spain divided by municipalities
spanish_mun = gpd.read_file(data_dir_geom + 'Municipios_IGN.geojson')
spanish_mun

,FID,INSPIREID,NATCODE,NAMEUNIT,CODNUT1,CODNUT2,CODNUT3,CODIGOINE,SHAPE_Length,SHAPE_Area,geometry
0,1,ES.IGN.SIGLIM34081616266,34081616266,Villarejo-Periesteban,ES4,ES42,ES423,16266,0.269748,0.003520,"MULTIPOLYGON (((-2.47791 39.88027, -2.47793 39..."
1,2,ES.IGN.SIGLIM34081616269,34081616269,Villares del Saz,ES4,ES42,ES423,16269,0.447608,0.007382,"MULTIPOLYGON (((-2.58669 39.85793, -2.58601 39..."
2,3,ES.IGN.SIGLIM34081616270,34081616270,Villarrubio,ES4,ES42,ES423,16270,0.305394,0.002978,"MULTIPOLYGON (((-2.96423 39.95773, -2.96231 39..."
3,4,ES.IGN.SIGLIM34081616271,34081616271,Villarta,ES4,ES42,ES423,16271,0.283123,0.002680,"MULTIPOLYGON (((-1.68041 39.46783, -1.67391 39..."
4,5,ES.IGN.SIGLIM34081616272,34081616272,Villas de la Ventosa,ES4,ES42,ES423,16272,0.595828,0.015355,"MULTIPOLYGON (((-2.49911 40.26601, -2.49878 40..."
...,...,...,...,...,...,...,...,...,...,...,...
8200,8207,ES.IGN.SIGLIM34053838050,34053838050,Vallehermoso,ES7,ES70,ES706,38050,0.788057,0.009971,"MULTIPOLYGON (((-17.32958 28.08035, -17.32959 ..."
8201,8208,ES.IGN.SIGLIM34053838051,34053838051,La Victoria de Acentejo,ES7,ES70,ES709,38051,0.210977,0.001666,"MULTIPOLYGON (((-16.48783 28.43752, -16.48775 ..."
8202,8209,ES.IGN.SIGLIM34053838052,34053838052,Vilaflor de Chasna,ES7,ES70,ES709,38052,0.330753,0.005170,"MULTIPOLYGON (((-16.68464 28.12064, -16.68468 ..."
8203,8210,ES.IGN.SIGLIM34053838053,34053838053,Villa de Mazo,ES7,ES70,ES707,38053,0.448598,0.006518,"MULTIPOLYGON (((-17.79311 28.52490, -17.79311 ..."


Considering the structure of this dataset, we can see the following:

- According to the INE, Spain has a total of 8132 municipalities. Although in this case a few more rows are observed, the number seems to be very close to the official one (https://www.ine.es/daco/daco42/codmun/cod_num_muni_provincia_ccaa.htm). This small difference is not a problem because we will make a previous filtering of municipalities by selected province and we will deal with those geometries that should not be there in case they fall in the selected province.

- Of all the columns, only three provide information:

  - **NAMEUNIT**: name of the municipality. This will allow us to know if they are the same as those indicated in the population dataset.

  - **CODIGOINE**: this is the code of the municipality. The first two numbers indicate the province to which they belong, which will allow us to carry out the desired filtering.

  - **geometry**: it is the key column to know whether or not a given establishment falls within a municipality.

With this in mind, we will be left with only these three columns.

In [ ]:
df_spanish_mun = spanish_mun[['NAMEUNIT', 'CODIGOINE', 'geometry']]
df_spanish_mun

,NAMEUNIT,CODIGOINE,geometry
0,Villarejo-Periesteban,16266,"MULTIPOLYGON (((-2.47791 39.88027, -2.47793 39..."
1,Villares del Saz,16269,"MULTIPOLYGON (((-2.58669 39.85793, -2.58601 39..."
2,Villarrubio,16270,"MULTIPOLYGON (((-2.96423 39.95773, -2.96231 39..."
3,Villarta,16271,"MULTIPOLYGON (((-1.68041 39.46783, -1.67391 39..."
4,Villas de la Ventosa,16272,"MULTIPOLYGON (((-2.49911 40.26601, -2.49878 40..."
...,...,...,...
8200,Vallehermoso,38050,"MULTIPOLYGON (((-17.32958 28.08035, -17.32959 ..."
8201,La Victoria de Acentejo,38051,"MULTIPOLYGON (((-16.48783 28.43752, -16.48775 ..."
8202,Vilaflor de Chasna,38052,"MULTIPOLYGON (((-16.68464 28.12064, -16.68468 ..."
8203,Villa de Mazo,38053,"MULTIPOLYGON (((-17.79311 28.52490, -17.79311 ..."


Next, we will create a dataset for each of the selected provinces. It will be on these datasets where we will examine the correlation between the variables mentioned at the beginning of this study. We will start with the municipalities of **Madrid**.

### **MADRID**

In [ ]:
# Read the file containing the population data by municipality
df_pop_Madrid = pd.read_excel(data_dir_INE + 'Municipality/Madrid.xlsx')
df_pop_Madrid

,municipality,code,population
0,"Acebeda, La",28001,67
1,Ajalvir,28002,4779
2,Alameda del Valle,28003,274
3,"Álamo, El",28004,10322
4,Alcalá de Henares,28005,199184
...,...,...,...
174,Villar del Olmo,28179,2258
175,Villarejo de Salvanés,28180,7746
176,Villaviciosa de Odón,28181,28750
177,Villavieja del Lozoya,28182,314


As we can see, this dataset is quite simple. We can see the name of the municipality, its code and the population in each one of them. Knowing that Madrid's code is 28, we will only keep the geometries of its municipalities.

In [ ]:
df_Madrid_mun = df_spanish_mun[df_spanish_mun['CODIGOINE'].str.startswith('28')]
df_Madrid_mun

,NAMEUNIT,CODIGOINE,geometry
395,La Acebeda,28001,"MULTIPOLYGON (((-3.67606 41.08670, -3.67503 41..."
396,Ajalvir,28002,"MULTIPOLYGON (((-3.51222 40.53864, -3.51113 40..."
397,Alameda del Valle,28003,"MULTIPOLYGON (((-3.89864 40.95883, -3.89518 40..."
398,El Álamo,28004,"MULTIPOLYGON (((-4.02403 40.24897, -4.01980 40..."
399,Alcalá de Henares,28005,"MULTIPOLYGON (((-3.44720 40.44631, -3.44426 40..."
...,...,...,...
569,Villavieja del Lozoya,28182,"MULTIPOLYGON (((-3.75554 41.02793, -3.75325 41..."
570,Zarzalejo,28183,"MULTIPOLYGON (((-4.19681 40.55903, -4.19571 40..."
571,Lozoyuela-Navas-Sieteiglesias,28901,"MULTIPOLYGON (((-3.67533 40.89220, -3.67471 40..."
572,Puentes Viejas,28902,"MULTIPOLYGON (((-3.62938 40.96155, -3.62811 40..."


As can be seen, the number of provinces coincides in both datasets. Now what we have to do is to count the number of establishments in each of these municipalities.

In [ ]:
poly_dict = {}

for i in df_Madrid_mun.index:

  poly_dict[df_Madrid_mun['NAMEUNIT'][i]] = df_Madrid_mun['geometry'][i]

polygons = gpd.GeoSeries(poly_dict)
polygons

La Acebeda                       MULTIPOLYGON (((-3.67606 41.08670, -3.67503 41...
Ajalvir                          MULTIPOLYGON (((-3.51222 40.53864, -3.51113 40...
Alameda del Valle                MULTIPOLYGON (((-3.89864 40.95883, -3.89518 40...
El Álamo                         MULTIPOLYGON (((-4.02403 40.24897, -4.01980 40...
Alcalá de Henares                MULTIPOLYGON (((-3.44720 40.44631, -3.44426 40...
                                                       ...                        
Villavieja del Lozoya            MULTIPOLYGON (((-3.75554 41.02793, -3.75325 41...
Zarzalejo                        MULTIPOLYGON (((-4.19681 40.55903, -4.19571 40...
Lozoyuela-Navas-Sieteiglesias    MULTIPOLYGON (((-3.67533 40.89220, -3.67471 40...
Puentes Viejas                   MULTIPOLYGON (((-3.62938 40.96155, -3.62811 40...
Tres Cantos                      MULTIPOLYGON (((-3.81051 40.60947, -3.81063 40...
Length: 179, dtype: geometry

In [ ]:
points_tourism = gpd.GeoDataFrame(index=estab_cleaned['id'], crs='epsg:4326', geometry= list(estab_cleaned['geometry']))

In [ ]:
# Check in which municipality each establishment of the dataset is located
estab_Madrid = points_tourism.assign(**{key: points_tourism.within(geom) for key, geom in polygons.items()})
estab_Madrid

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

,geometry,La Acebeda,Ajalvir,Alameda del Valle,El Álamo,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,...,Villanueva del Pardillo,Villanueva de Perales,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo,Lozoyuela-Navas-Sieteiglesias,Puentes Viejas,Tres Cantos
id,,,,,,,,,,,,,,,,,,,,,
25913327,POINT (-3.78818 40.39844),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860899,POINT (1.35784 42.37309),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860903,POINT (3.28194 42.28806),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860947,POINT (-1.85918 37.14083),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860948,POINT (-5.17184 36.72109),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446745740,POINT (2.15293 41.38762),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2453486016,POINT (-1.11629 38.60157),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2457900078,POINT (-7.70224 42.69472),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
estab_Madrid_number = pd.DataFrame()

for column in estab_Madrid:

  if column != 'geometry':

    number = np.count_nonzero(estab_Madrid[column] == True)

    new_row = {'municipality':column, 'establishments':number}

    estab_Madrid_number = pd.concat([estab_Madrid_number, pd.DataFrame([new_row])], ignore_index=True)

estab_Madrid_number

,municipality,establishments
0,La Acebeda,2
1,Ajalvir,3
2,Alameda del Valle,4
3,El Álamo,0
4,Alcalá de Henares,27
...,...,...
174,Villavieja del Lozoya,1
175,Zarzalejo,1
176,Lozoyuela-Navas-Sieteiglesias,0
177,Puentes Viejas,6


In [ ]:
estab_Madrid_number.sort_values(by='establishments', ascending=False)

,municipality,establishments
77,Madrid,791
4,Alcalá de Henares,27
64,Getafe,20
114,Rascafría,16
140,Torrejón de Ardoz,14
...,...,...
112,Puebla de la Sierra,0
111,Prádena del Rincón,0
110,Pozuelo del Rey,0
106,Pinilla del Valle,0


As expected, the municipality with the largest number of tourist establishments is Madrid (791), followed by Alcalá de Henares (27) and Getafe (20).

Once we have obtained the tourist establishments for each municipality, we will proceed to create our final dataset.

In [ ]:
# Add establishments
df_final_Madrid = df_Madrid_mun.copy()

df_final_Madrid = df_final_Madrid.rename(columns={'NAMEUNIT':'municipality', 'CODIGOINE': 'code'})

df_final_Madrid = df_final_Madrid.merge(estab_Madrid_number, how='inner', on='municipality')
df_final_Madrid

,municipality,code,geometry,establishments
0,La Acebeda,28001,"MULTIPOLYGON (((-3.67606 41.08670, -3.67503 41...",2
1,Ajalvir,28002,"MULTIPOLYGON (((-3.51222 40.53864, -3.51113 40...",3
2,Alameda del Valle,28003,"MULTIPOLYGON (((-3.89864 40.95883, -3.89518 40...",4
3,El Álamo,28004,"MULTIPOLYGON (((-4.02403 40.24897, -4.01980 40...",0
4,Alcalá de Henares,28005,"MULTIPOLYGON (((-3.44720 40.44631, -3.44426 40...",27
...,...,...,...,...
174,Villavieja del Lozoya,28182,"MULTIPOLYGON (((-3.75554 41.02793, -3.75325 41...",1
175,Zarzalejo,28183,"MULTIPOLYGON (((-4.19681 40.55903, -4.19571 40...",1
176,Lozoyuela-Navas-Sieteiglesias,28901,"MULTIPOLYGON (((-3.67533 40.89220, -3.67471 40...",0
177,Puentes Viejas,28902,"MULTIPOLYGON (((-3.62938 40.96155, -3.62811 40...",6


In [ ]:
df_final_Madrid['code'] = df_final_Madrid['code'].astype(str).astype(int)

In [ ]:
# Add population
df_final_Madrid = df_final_Madrid.merge(df_pop_Madrid, how='inner', on='code')
df_final_Madrid

,municipality_x,code,geometry,establishments,municipality_y,population
0,La Acebeda,28001,"MULTIPOLYGON (((-3.67606 41.08670, -3.67503 41...",2,"Acebeda, La",67
1,Ajalvir,28002,"MULTIPOLYGON (((-3.51222 40.53864, -3.51113 40...",3,Ajalvir,4779
2,Alameda del Valle,28003,"MULTIPOLYGON (((-3.89864 40.95883, -3.89518 40...",4,Alameda del Valle,274
3,El Álamo,28004,"MULTIPOLYGON (((-4.02403 40.24897, -4.01980 40...",0,"Álamo, El",10322
4,Alcalá de Henares,28005,"MULTIPOLYGON (((-3.44720 40.44631, -3.44426 40...",27,Alcalá de Henares,199184
...,...,...,...,...,...,...
174,Villavieja del Lozoya,28182,"MULTIPOLYGON (((-3.75554 41.02793, -3.75325 41...",1,Villavieja del Lozoya,314
175,Zarzalejo,28183,"MULTIPOLYGON (((-4.19681 40.55903, -4.19571 40...",1,Zarzalejo,1810
176,Lozoyuela-Navas-Sieteiglesias,28901,"MULTIPOLYGON (((-3.67533 40.89220, -3.67471 40...",0,Lozoyuela-Navas-Sieteiglesias,1483
177,Puentes Viejas,28902,"MULTIPOLYGON (((-3.62938 40.96155, -3.62811 40...",6,Puentes Viejas,760


In [ ]:
# Obtain the monthly number of tourists received in each municipality
df_Madrid = df_tourists[df_tourists['prov_dest'] == 'Madrid']
df_Madrid =  df_Madrid[df_Madrid['pais_orig'] == 'Total']
df_Madrid

,mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,prov_dest_cod,prov_dest
12761,2023-01,0,Total,28002,Ajalvir,182,28,Madrid
12765,2023-01,0,Total,28005,Alcalá de Henares,7381,28,Madrid
12804,2023-01,0,Total,28006,Alcobendas,5873,28,Madrid
12839,2023-01,0,Total,28007,Alcorcón,3389,28,Madrid
12869,2023-01,0,Total,28008,Aldea del Fresno,101,28,Madrid
...,...,...,...,...,...,...,...,...
17717,2023-12,0,Total,28181,Villaviciosa de Odón,2853,28,Madrid
17740,2023-12,0,Total,28183,Zarzalejo,72,28,Madrid
17743,2023-12,0,Total,28901,Lozoyuela-Navas-Sieteiglesias,157,28,Madrid
17746,2023-12,0,Total,28902,Puentes Viejas,386,28,Madrid


In [ ]:
## Obtain average number of tourists received in each municipality during 2023
df_mean_Madrid = df_Madrid.groupby(['mun_dest','mun_dest_cod'])['turistas'].mean().round(0)
df_mean_Madrid = df_mean_Madrid.to_frame().reset_index()
df_mean_Madrid

,mun_dest,mun_dest_cod,turistas
0,Ajalvir,28002,201.0
1,Alcalá de Henares,28005,9144.0
2,Alcobendas,28006,7654.0
3,Alcorcón,28007,4087.0
4,Aldea del Fresno,28008,113.0
...,...,...,...
160,Villar del Olmo,28179,102.0
161,Villarejo de Salvanés,28180,348.0
162,Villaviciosa de Odón,28181,2299.0
163,Zarzalejo,28183,69.0


In [ ]:
df_mean_Madrid = df_mean_Madrid.rename(columns={'mun_dest_cod':'code', 'turistas': 'tourists'})
df_mean_Madrid['code'] = df_mean_Madrid['code'].astype(str).astype(int)
df_mean_Madrid

,mun_dest,code,tourists
0,Ajalvir,28002,201.0
1,Alcalá de Henares,28005,9144.0
2,Alcobendas,28006,7654.0
3,Alcorcón,28007,4087.0
4,Aldea del Fresno,28008,113.0
...,...,...,...
160,Villar del Olmo,28179,102.0
161,Villarejo de Salvanés,28180,348.0
162,Villaviciosa de Odón,28181,2299.0
163,Zarzalejo,28183,69.0


In [ ]:
# Add number of tourists
df_final_Madrid = df_final_Madrid.merge(df_mean_Madrid, how='inner', on='code')
df_final_Madrid

,municipality_x,code,geometry,establishments,municipality_y,population,mun_dest,tourists
0,Ajalvir,28002,"MULTIPOLYGON (((-3.51222 40.53864, -3.51113 40...",3,Ajalvir,4779,Ajalvir,201.0
1,El Álamo,28004,"MULTIPOLYGON (((-4.02403 40.24897, -4.01980 40...",0,"Álamo, El",10322,"Álamo, El",60.0
2,Alcalá de Henares,28005,"MULTIPOLYGON (((-3.44720 40.44631, -3.44426 40...",27,Alcalá de Henares,199184,Alcalá de Henares,9144.0
3,Alcobendas,28006,"MULTIPOLYGON (((-3.67414 40.58886, -3.67268 40...",10,Alcobendas,119416,Alcobendas,7654.0
4,Alcorcón,28007,"MULTIPOLYGON (((-3.87092 40.35312, -3.87097 40...",8,Alcorcón,171772,Alcorcón,4087.0
...,...,...,...,...,...,...,...,...
160,Villaviciosa de Odón,28181,"MULTIPOLYGON (((-4.00639 40.33939, -4.00622 40...",7,Villaviciosa de Odón,28750,Villaviciosa de Odón,2299.0
161,Zarzalejo,28183,"MULTIPOLYGON (((-4.19681 40.55903, -4.19571 40...",1,Zarzalejo,1810,Zarzalejo,69.0
162,Lozoyuela-Navas-Sieteiglesias,28901,"MULTIPOLYGON (((-3.67533 40.89220, -3.67471 40...",0,Lozoyuela-Navas-Sieteiglesias,1483,Lozoyuela-Navas-Sieteiglesias,187.0
163,Puentes Viejas,28902,"MULTIPOLYGON (((-3.62938 40.96155, -3.62811 40...",6,Puentes Viejas,760,Puentes Viejas,257.0


To finish with the creation of our dataset it would simply be necessary to eliminate the columns that are not useful and rename some of those that remain.

It is important to note that in the case of Madrid, since tourism data for all the municipalities are not available (when calculating the average, the dataframe only has 165 lines, not 179), its data are not taken into account in the final dataset since it was "lost" when performing the merge.

In [ ]:
# Drop some columns
df_final_Madrid = df_final_Madrid.drop(['code', 'geometry', 'municipality_y', 'mun_dest'], axis=1)

# Rename columns
df_final_Madrid = df_final_Madrid.rename(columns={'municipality_x':'municipality'})

However, before we move on to the correlations section, we are going to carry out a small feature engineering process in order to add our main variable for this study: the ratio **number of tourists/number of establishments**. This variable will allow us to know the approximate number of tourists staying in a tourist establishment in each municipality.

Note: **The number of tourists used to calculate the ratio is an average of the tourists received in each municipality throughout 2023, not the average number of tourists staying in establishments. Since what we want to check with this variable is the number of tourists that approximately corresponds to a tourist accommodation in each municipality, it is possible that the figure obtained does not accurately reflect the reality.**



In [ ]:
# Compute and add ratio
tour_estab = []

for i in range(len(df_final_Madrid)):

  if df_final_Madrid.iloc[i]['establishments'] == 0:

    ratio = 0
    tour_estab.append(ratio)

  else:

    ratio = df_final_Madrid.iloc[i]['tourists']/df_final_Madrid.iloc[i]['establishments']
    tour_estab.append(ratio)

df_final_Madrid['tourists/establishments'] = list(tour_estab)

df_final_Madrid

,municipality,establishments,population,tourists,tourists/establishments
0,Ajalvir,3,4779,201.0,67.000000
1,El Álamo,0,10322,60.0,0.000000
2,Alcalá de Henares,27,199184,9144.0,338.666667
3,Alcobendas,10,119416,7654.0,765.400000
4,Alcorcón,8,171772,4087.0,510.875000
...,...,...,...,...,...
160,Villaviciosa de Odón,7,28750,2299.0,328.428571
161,Zarzalejo,1,1810,69.0,69.000000
162,Lozoyuela-Navas-Sieteiglesias,0,1483,187.0,0.000000
163,Puentes Viejas,6,760,257.0,42.833333


We can now start to check the relationships between our variables.

## 3. Correlations

The first thing to know is that correlation coefficients quantify the association between variables or features of a dataset. We can find three different forms of correlation:

1. **Negative correlation.** In a plot, the y values tend to decrease as the x values increase. That is, large values of one feature correspond to small values of the other, and vice versa.

2. **Weak or no correlation.** There is no obvious trend. This  occurs when an association between two features is not obvious or is hardly observable.

3. **Positive correlation.** In a plot, the y values tend to increase as the x values increase. In other words, large values of one feature correspond to large values of the other, and vice versa.

Therefore, to check the correlation between the different variables in our dataset, we are going to use the Pandas *corr()* method. This function allows us to calculate the pairwise correlation between all the columns in the dataset. Although there are different correlation coefficients, in this study we will focus on **Pearson's** coefficient, which is a measure of linear correlation between two variables.

In [ ]:
df_corr = df_final_Madrid.corr(numeric_only = True)
df_corr.style.background_gradient(cmap='coolwarm')

,establishments,population,tourists,tourists/establishments
establishments,1.000000,0.991064,0.998828,0.103691
population,0.991064,1.000000,0.992294,0.190463
tourists,0.998828,0.992294,1.000000,0.117892
tourists/establishments,0.103691,0.190463,0.117892,1.000000


As we can see, there is a strong positive correlation between the number of tourist establishments, the number of tourists and the population of a municipality. On the other hand, there is hardly any correlation between our ratio and the rest of the variables. All this is explained as follows:

- The greater the number of tourists, the greater the tourist offer. That is to say, if the number of tourists visiting a municipality is high, their economic investment will be greater compared to other areas, and businesspeople will be interested in opening new establishments in those places. If we think about it the other way around, it makes sense that the greater the number of tourist establishments in a municipality, the greater the number of tourists received.  From my point of view, this could be due to two main reasons:

    - If there are more accommodation options in a location, the likelihood of renting a room in one of those locations will be higher. For example, if we are looking for a hotel in Madrid, it is more likely that we will first find options in the center, which is where there are more tourist accommodations, than in one of the peripheral regions.

    - The fact that the number of accommodations is higher will also be an indication that tourists prefer that area, so other tourists will end up doing the same and visiting the most popular places. In other words, tourists attract more tourists.

  However, it is important to keep in mind that there could be exceptions in which new businesses could not be opened in a very touristic area due to different reasons (protected spaces, legislation in the area...) or the tourist demand is so high that it cannot be completely covered even if new establishments are opened.

- In the case of the relationship between tourists and population, the same is usually true. If a municipality has a high population, it means that it is attractive to live in, as is the case with Madrid downtown. The more people live in a place, the more services and activities are offered, which will also make it attractive to people from other areas. Therefore, it makes sense that the higher the population, the greater the number of arriving tourists. However, there may also be municipalities where the population is much smaller due to their size, but they are equally attractive for tourism, or the opposite case. In other words, there is a relationship between the two variables and, in general, it makes sense that if one grows, the other will grow as well.

- As for the last positive relationship we have, there is not much left to explain. The greater the population, the greater the number of tourist establishments. The economic investment of both groups, inhabitants and visitors, increases the probability that an entrepreneur will want to open a new business in the area.

- Finally, we see that our new variable has hardly any relationship with the original ones. Since this ratio is calculated by dividing tourists by establishments, it is clear that its value should be directly proportional to the number of tourists (if tourists increase, so will the ratio -> positive correlation) and inversely proportional to the number of establishments (the larger the denominator, the smaller the ratio -> negative correlation). However, we have seen that these two variables have a very strong positive correlation; that is, if one grows, the other should also grow. This behavior is what causes the positive correlation to cancel out with the negative correlation, so that there is hardly any correlation between these two variables and the ratio. We do see a slightly higher correlation with population, which makes sense because it does not participate in the calculation of the ratio. Even so, there does not seem to be a relationship between these two variables, which makes sense since the population also has a strong positive correlation with the other two variables. In other words, if the population grows, generally so will the other two variables, and the ratio will increase in both its numerator and denominator. This result also allows us to rule out the possibility that one of the two variables in the ratio grows much more with respect to the other (this can be checked with a linear regression).

We will now repeat this entire process for the rest of the selected provinces. This will allow us to check if the behavior of the variables coincides in all of them. The provinces selected will be the same as those used in other notebooks: Valencia (coastal), Illes Balears (islands) and Jaén (province with low population). Since the procedure has already been explained in previous steps, we will limit ourselves to commenting on the results obtained for each of the provinces.

### **SELECTED PROVINCE**

To avoid copying and pasting the code as many times as provinces we analyze, we will create two variables that can be easily modified. When commenting on the results, we will write the name of the province just above them.

In [ ]:
# province: Valencia, Illes_Balears, Jaen
# code: 46, 07, 23
province = 'Jaen'
code = '23'

In [ ]:
# Read the file containing the population data by municipality
df_pop = pd.read_excel(data_dir_INE + 'Municipality/' + province + '.xlsx')
df_pop

,municipality,code,population
0,Albanchez de Mágina,23001,958
1,Alcalá la Real,23002,21587
2,Alcaudete,23003,10265
3,Aldeaquemada,23004,468
4,Andújar,23005,35788
...,...,...,...
92,Villanueva del Arzobispo,23097,7933
93,Villardompardo,23098,920
94,"Villares, Los",23099,6079
95,Villarrodrigo,23101,383


### **Valencia**

As can be seen, Valencia has 266 municipalities.

### **Illes Balears**

As can be seen, Illes Balears has 67 municipalities.

### **Jaén**

As can be seen, Jaén has 97 municipalities.

In [ ]:
df_province_mun = df_spanish_mun[df_spanish_mun['CODIGOINE'].str.startswith(code)]
df_province_mun

,NAMEUNIT,CODIGOINE,geometry
3263,Albanchez de Mágina,23001,"MULTIPOLYGON (((-3.48351 37.80497, -3.48306 37..."
3264,Alcalá la Real,23002,"MULTIPOLYGON (((-4.08735 37.48593, -4.08765 37..."
3265,Alcaudete,23003,"MULTIPOLYGON (((-4.21818 37.62654, -4.21825 37..."
3266,Aldeaquemada,23004,"MULTIPOLYGON (((-3.45316 38.40020, -3.45170 38..."
3267,Andújar,23005,"MULTIPOLYGON (((-4.26887 38.34717, -4.26910 38..."
...,...,...,...
3360,Cárcheles,23901,"MULTIPOLYGON (((-3.67171 37.66765, -3.67071 37..."
3361,Bedmar y Garcíez,23902,"MULTIPOLYGON (((-3.51081 37.86850, -3.51021 37..."
3362,Villatorres,23903,"MULTIPOLYGON (((-3.77602 37.92152, -3.77660 37..."
3363,Santiago-Pontones,23904,"MULTIPOLYGON (((-2.90413 38.02368, -2.90407 38..."


### **Valencia**

If we select from the total number of geometries those belonging to the province of Valencia (code 46), we see that 266 also come out.

### **Illes Balears**

If we select from the total number of geometries those belonging to the province of Illes Balears (code 07), we see that 67 also come out.

### **Jaén**

If we select from the total number of geometries those belonging to the province of Jaén (code 23), we see that 97 also come out.

In [ ]:
poly_dict = {}

for i in df_province_mun.index:

  poly_dict[df_province_mun['NAMEUNIT'][i]] = df_province_mun['geometry'][i]

polygons = gpd.GeoSeries(poly_dict)
polygons

Albanchez de Mágina    MULTIPOLYGON (((-3.48351 37.80497, -3.48306 37...
Alcalá la Real         MULTIPOLYGON (((-4.08735 37.48593, -4.08765 37...
Alcaudete              MULTIPOLYGON (((-4.21818 37.62654, -4.21825 37...
Aldeaquemada           MULTIPOLYGON (((-3.45316 38.40020, -3.45170 38...
Andújar                MULTIPOLYGON (((-4.26887 38.34717, -4.26910 38...
                                             ...                        
Cárcheles              MULTIPOLYGON (((-3.67171 37.66765, -3.67071 37...
Bedmar y Garcíez       MULTIPOLYGON (((-3.51081 37.86850, -3.51021 37...
Villatorres            MULTIPOLYGON (((-3.77602 37.92152, -3.77660 37...
Santiago-Pontones      MULTIPOLYGON (((-2.90413 38.02368, -2.90407 38...
Arroyo del Ojanco      MULTIPOLYGON (((-2.95987 38.28949, -2.95957 38...
Length: 97, dtype: geometry

In [ ]:
points_tourism_mun = gpd.GeoDataFrame(index=estab_cleaned['id'], crs='epsg:4326', geometry= list(estab_cleaned['geometry']))

In [ ]:
# Check in which municipality each establishment of the dataset is located
estab_mun = points_tourism_mun.assign(**{key: points_tourism_mun.within(geom) for key, geom in polygons.items()})
estab_mun

,geometry,Albanchez de Mágina,Alcalá la Real,Alcaudete,Aldeaquemada,Andújar,Arjona,Arjonilla,Arquillos,Baeza,...,Villanueva de la Reina,Villanueva del Arzobispo,Villardompardo,Los Villares,Villarrodrigo,Cárcheles,Bedmar y Garcíez,Villatorres,Santiago-Pontones,Arroyo del Ojanco
id,,,,,,,,,,,,,,,,,,,,,
25913327,POINT (-3.78818 40.39844),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860899,POINT (1.35784 42.37309),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860903,POINT (3.28194 42.28806),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860947,POINT (-1.85918 37.14083),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26860948,POINT (-5.17184 36.72109),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446745740,POINT (2.15293 41.38762),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2453486016,POINT (-1.11629 38.60157),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2457900078,POINT (-7.70224 42.69472),False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
estab_number = pd.DataFrame()

for column in estab_mun:

  if column != 'geometry':

    number = np.count_nonzero(estab_mun[column] == True)

    new_row = {'municipality':column, 'establishments':number}

    estab_number = pd.concat([estab_number, pd.DataFrame([new_row])], ignore_index=True)

estab_number

,municipality,establishments
0,Albanchez de Mágina,1
1,Alcalá la Real,9
2,Alcaudete,2
3,Aldeaquemada,1
4,Andújar,8
...,...,...
92,Cárcheles,5
93,Bedmar y Garcíez,2
94,Villatorres,1
95,Santiago-Pontones,24


In [ ]:
estab_number.sort_values(by='establishments', ascending=False)

,municipality,establishments
24,Cazorla,32
42,La Iruela,25
45,Jaén,25
95,Santiago-Pontones,24
83,Úbeda,23
...,...,...
35,Lahiguera,0
71,Santiago de Calatrava,0
32,Génave,0
31,Fuerte del Rey,0


### **Valencia**

As expected, the municipality with the largest number of tourist establishments is València (145), followed by Gandia (33) and Oliva (23).

### **Illes Balears**

In this case, it is not the capital of the largest island the one with the highest number of establishments, but it is in the Top 3. The three municipalities with the highest number of tourist accommodations are Calvià (201), Palma (200) y Pollença (159).


### **Jaén**

As we can see, Jaén is the province whose municipalities have the lowest number of tourist establishments. Specifically, Cazorla (32), La Iruela (25) and Jaén (25) stand out, although the numbers do not differ much between them.

Once we have obtained the tourist establishments for each municipality, we will proceed to create our final dataset.

In [ ]:
# Add establishments
df_final = df_province_mun.copy()

df_final = df_final.rename(columns={'NAMEUNIT':'municipality', 'CODIGOINE': 'code'})

df_final = df_final.merge(estab_number, how='inner', on='municipality')
df_final

,municipality,code,geometry,establishments
0,Albanchez de Mágina,23001,"MULTIPOLYGON (((-3.48351 37.80497, -3.48306 37...",1
1,Alcalá la Real,23002,"MULTIPOLYGON (((-4.08735 37.48593, -4.08765 37...",9
2,Alcaudete,23003,"MULTIPOLYGON (((-4.21818 37.62654, -4.21825 37...",2
3,Aldeaquemada,23004,"MULTIPOLYGON (((-3.45316 38.40020, -3.45170 38...",1
4,Andújar,23005,"MULTIPOLYGON (((-4.26887 38.34717, -4.26910 38...",8
...,...,...,...,...
92,Cárcheles,23901,"MULTIPOLYGON (((-3.67171 37.66765, -3.67071 37...",5
93,Bedmar y Garcíez,23902,"MULTIPOLYGON (((-3.51081 37.86850, -3.51021 37...",2
94,Villatorres,23903,"MULTIPOLYGON (((-3.77602 37.92152, -3.77660 37...",1
95,Santiago-Pontones,23904,"MULTIPOLYGON (((-2.90413 38.02368, -2.90407 38...",24


In [ ]:
df_final['code'] = df_final['code'].astype(str).astype(int)

In [ ]:
# Add population
df_final = df_final.merge(df_pop, how='inner', on='code')
df_final

,municipality_x,code,geometry,establishments,municipality_y,population
0,Albanchez de Mágina,23001,"MULTIPOLYGON (((-3.48351 37.80497, -3.48306 37...",1,Albanchez de Mágina,958
1,Alcalá la Real,23002,"MULTIPOLYGON (((-4.08735 37.48593, -4.08765 37...",9,Alcalá la Real,21587
2,Alcaudete,23003,"MULTIPOLYGON (((-4.21818 37.62654, -4.21825 37...",2,Alcaudete,10265
3,Aldeaquemada,23004,"MULTIPOLYGON (((-3.45316 38.40020, -3.45170 38...",1,Aldeaquemada,468
4,Andújar,23005,"MULTIPOLYGON (((-4.26887 38.34717, -4.26910 38...",8,Andújar,35788
...,...,...,...,...,...,...
92,Cárcheles,23901,"MULTIPOLYGON (((-3.67171 37.66765, -3.67071 37...",5,Cárcheles,1300
93,Bedmar y Garcíez,23902,"MULTIPOLYGON (((-3.51081 37.86850, -3.51021 37...",2,Bedmar y Garcíez,2605
94,Villatorres,23903,"MULTIPOLYGON (((-3.77602 37.92152, -3.77660 37...",1,Villatorres,4261
95,Santiago-Pontones,23904,"MULTIPOLYGON (((-2.90413 38.02368, -2.90407 38...",24,Santiago-Pontones,2729


In [ ]:
# Obtain the monthly number of tourists received in each municipality
# Valencia = Valencia/València, Illes Balears = Balears, Illes, Jaen = Jaén
prov_tourists = 'Jaén'
df_province = df_tourists[df_tourists['prov_dest'] == prov_tourists]
df_province =  df_province[df_province['pais_orig'] == 'Total']
df_province

,mes,pais_orig_cod,pais_orig,mun_dest_cod,mun_dest,turistas,prov_dest_cod,prov_dest
11338,2023-01,0,Total,23002,Alcalá la Real,437,23,Jaén
11345,2023-01,0,Total,23003,Alcaudete,246,23,Jaén
11351,2023-01,0,Total,23005,Andújar,664,23,Jaén
11360,2023-01,0,Total,23006,Arjona,86,23,Jaén
11363,2023-01,0,Total,23009,Baeza,344,23,Jaén
...,...,...,...,...,...,...,...,...
14485,2023-12,0,Total,23901,Cárcheles,99,23,Jaén
14488,2023-12,0,Total,23902,Bedmar y Garcíez,75,23,Jaén
14491,2023-12,0,Total,23903,Villatorres,35,23,Jaén
14493,2023-12,0,Total,23904,Santiago-Pontones,228,23,Jaén


In [ ]:
## Obtain average number of tourists received in each municipality during 2023
df_mean = df_province.groupby(['mun_dest','mun_dest_cod'])['turistas'].mean().round(0)
df_mean = df_mean.to_frame().reset_index()
df_mean

,mun_dest,mun_dest_cod,turistas
0,Alcalá la Real,23002,673.0
1,Alcaudete,23003,401.0
2,Andújar,23005,736.0
3,Arjona,23006,82.0
4,Arjonilla,23007,42.0
...,...,...,...
72,Villanueva del Arzobispo,23097,165.0
73,"Villares, Los",23099,80.0
74,Villarrodrigo,23101,34.0
75,Villatorres,23903,33.0


In [ ]:
df_mean = df_mean.rename(columns={'mun_dest_cod':'code', 'turistas': 'tourists'})
df_mean['code'] = df_mean['code'].astype(str).astype(int)
df_mean

,mun_dest,code,tourists
0,Alcalá la Real,23002,673.0
1,Alcaudete,23003,401.0
2,Andújar,23005,736.0
3,Arjona,23006,82.0
4,Arjonilla,23007,42.0
...,...,...,...
72,Villanueva del Arzobispo,23097,165.0
73,"Villares, Los",23099,80.0
74,Villarrodrigo,23101,34.0
75,Villatorres,23903,33.0


In [ ]:
# Add number of tourists
df_final = df_final.merge(df_mean, how='inner', on='code')
df_final

,municipality_x,code,geometry,establishments,municipality_y,population,mun_dest,tourists
0,Alcalá la Real,23002,"MULTIPOLYGON (((-4.08735 37.48593, -4.08765 37...",9,Alcalá la Real,21587,Alcalá la Real,673.0
1,Alcaudete,23003,"MULTIPOLYGON (((-4.21818 37.62654, -4.21825 37...",2,Alcaudete,10265,Alcaudete,401.0
2,Andújar,23005,"MULTIPOLYGON (((-4.26887 38.34717, -4.26910 38...",8,Andújar,35788,Andújar,736.0
3,Arjona,23006,"MULTIPOLYGON (((-4.20627 37.98997, -4.20578 37...",1,Arjona,5349,Arjona,82.0
4,Arjonilla,23007,"MULTIPOLYGON (((-4.17260 37.98731, -4.17189 37...",2,Arjonilla,3539,Arjonilla,42.0
...,...,...,...,...,...,...,...,...
72,Cárcheles,23901,"MULTIPOLYGON (((-3.67171 37.66765, -3.67071 37...",5,Cárcheles,1300,Cárcheles,98.0
73,Bedmar y Garcíez,23902,"MULTIPOLYGON (((-3.51081 37.86850, -3.51021 37...",2,Bedmar y Garcíez,2605,Bedmar y Garcíez,60.0
74,Villatorres,23903,"MULTIPOLYGON (((-3.77602 37.92152, -3.77660 37...",1,Villatorres,4261,Villatorres,33.0
75,Santiago-Pontones,23904,"MULTIPOLYGON (((-2.90413 38.02368, -2.90407 38...",24,Santiago-Pontones,2729,Santiago-Pontones,269.0


In [ ]:
# Drop some columns
df_final = df_final.drop(['code', 'geometry', 'municipality_y', 'mun_dest'], axis=1)

# Rename columns
df_final = df_final.rename(columns={'municipality_x':'municipality'})

In [ ]:
# Compute and add ratio
tour_estab = []

for i in range(len(df_final)):

  if df_final.iloc[i]['establishments'] == 0:

    ratio = 0
    tour_estab.append(ratio)

  else:

    ratio = df_final.iloc[i]['tourists']/df_final.iloc[i]['establishments']
    tour_estab.append(ratio)

df_final['tourists/establishments'] = list(tour_estab)

df_final

,municipality,establishments,population,tourists,tourists/establishments
0,Alcalá la Real,9,21587,673.0,74.777778
1,Alcaudete,2,10265,401.0,200.500000
2,Andújar,8,35788,736.0,92.000000
3,Arjona,1,5349,82.0,82.000000
4,Arjonilla,2,3539,42.0,21.000000
...,...,...,...,...,...
72,Cárcheles,5,1300,98.0,19.600000
73,Bedmar y Garcíez,2,2605,60.0,30.000000
74,Villatorres,1,4261,33.0,33.000000
75,Santiago-Pontones,24,2729,269.0,11.208333


Once we have our final dataset, we can check the correlations between our variables.

In [ ]:
df_corr = df_final.corr(numeric_only = True)
df_corr.style.background_gradient(cmap='coolwarm')

,establishments,population,tourists,tourists/establishments
establishments,1.000000,0.393078,0.503715,-0.091007
population,0.393078,1.000000,0.920985,0.393602
tourists,0.503715,0.920985,1.000000,0.426156
tourists/establishments,-0.091007,0.393602,0.426156,1.000000


As we did before with Madrid, now we are going to check what values have been obtained in the correlation matrix for each of the chosen provinces.

### **Valencia**

*   tourists - establishments -> 0.949944
*   tourists - population -> 0.986942
*   establishments - population -> 0.951330

If we recall the values obtained for Madrid, we see that in Valencia the correlation between the variables is somewhat lower. Nevertheless, we still see a fairly strong positive correlation between the three original variables and hardly any correlation, although slightly higher than in the case of Madrid, between all these variables and the calculated ratio.


### **Illes Balears**

*   tourists - establishments -> 0.849194
*   tourists - population -> 0.845954
*   establishments - population -> 0.597326

In the case of this province, we see much different results than those obtained in the two previous scenarios. First, the correlation between our three original variables seems to decrease, especially between establishments and population. Although there is still a positive correlation, it is around 0.6, 0.3 below the results obtained for the other two provinces. This means that it is not always going to be the case that in areas where there is more population we will also find a greater number of tourist establishments. This could be due to the type of territory we are dealing with: an island especially focused on tourism. That is to say, on islands like Mallorca or Ibiza we can find large concentrations of accommodation for tourists. Areas especially focused on tourism whose price is very high. This implies that it will be more likely that a tourist will be willing to spend money to stay in those areas but an inhabitant will prefer a less touristic and cheaper area to live.

Although with a lower correlation, it is still true that the number of tourists is associated with a greater number of establishments, which makes a lot of sense, and with a larger population. In the first case, this somewhat lower correlation may be due to all the natural areas of the island. In these areas tourism will be very high, but it will be in other places where tourists will stay. In fact, it is very typical to rent a car in this type of destinations.

As for the second correlation, it is closely related to the above. It is true that, since most of the island is dedicated to tourism, the inhabitants live in very touristic areas, such as the old town, which in the past would have had much more affordable prices. Therefore, it makes sense that, in general, where there is more population is where more tourists go. The fact that this correlation is weaker could be due to natural areas such as coves, where the population is not as abundant as tourism. And, although it has been mentioned above, the areas where people live on the island do not usually correspond to the areas where tourist accommodation is concentrated (except for the capital, for example).

Finally, we see that the lower the correlation between these variables, the higher their correlation with the ratio. In fact, we can highlight the correlation of the ratio with the number of tourists, which is around 0.5. This is because the correlation between tourists and establishments is not so high, so that the two variables will not always grow and there could be cases in which tourists grow and establishments do not, thus producing an increase in the ratio.


### **Jaén**

*   tourists - establishments -> 0.503715
*   tourists - population -> 0.920985
*   establishments - population -> 0.393078

In the latter case, we observed the lowest correlations between some of our original variables. In the case of the tourist-population pair of variables, we do see a value very close to 1, so it is true that the areas where most people live are usually the most touristic ones. However, in comparison with the case of the islands, we see that the areas where tourist establishments predominate are not the most visited by tourists. In other words, visitors do not necessarily stay in the same places they are going to visit. Taking into account the above, we see that what happens is that the most visited areas are also the most populated, but that tourists do not stay in these areas, but in other much less touristy areas where it may be easier to open a tourist establishment.

As for the correlation of these three variables with the ratio, we can see that here it is possible to observe how this value can vary if the numerator or denominator increases or decreases. First, we see a slightly negative correlation between the number of establishments and the ratio. In other words, it is clear that if the denominator increases, the ratio will become smaller. On the other hand, although it is still a weak correlation, we do see that the correlation between the number of tourists and the ratio is somewhat higher compared to the previous provinces. This means that, if the numerator of a fraction increases, its value usually increases as well. As we have seen, the lower the correlation between tourists and establishments, the higher the correlation of these variables with our ratio (positively or negatively, respectively). Finally, we continue to see that there is hardly any correlation between the ratio and the population of a municipality.

Once this analysis has been completed, let us recall the correlation values obtained between these same variables but at the province level instead of the municipality level:

*   tourists - establishments -> 0.823425
*   tourists - population -> 0.645295
*   establishments - population -> 0.555035

As we can see, there is not a single province whose values coincide with those obtained at the province level for Spain as a whole. The one that could be most similar would be Illes Balears, although the correlation obtained in this province for the second pair of variables is approximately 0.2 higher. In other words, we can say that, with the exception of Jaén, in the rest of the provinces an increase in the number of tourists translates into an increase in tourist accommodations and vice versa. Of course, there may be exceptions, such as the space available to open new businesses, protected areas where construction is not allowed...

In the case of the correlation between tourists and population, we have already seen in some previous examples that tourists do not always visit the most populated areas. It is true that, in general, the most populated cities tend to be the most touristy places due to their history (in the capitals, for example, the number of monuments will be high) and the activities and services available, but there may also be areas that have gradually become tourist hotspots and that, on the contrary, and due to various reasons (protected natural areas, very high prices aimed at tourists, concentrations of tourist accommodation...), are not the places chosen by the usual population.

We can say that something similar happens with establishments and tourists. As we saw in the Balearic Islands, it is common to find places aimed exclusively at the accommodation of tourists, so it is unlikely that the permanent population lives in these high-priced areas. Even so, in the provincial capitals it is also common to find a high number of options for tourist accommodation.

Finally, if we consider the correlation between these three variables and the ratio, the results are practically the same as those obtained within each province: there is hardly any correlation.

Therefore, we draw the following conclusions:

  - Although each province has its own characteristics and, therefore, a different correlation between the three variables under study, they do reflect in some way the behavior of these variables throughout the country.

  - There is hardly any correlation between the three original variables and the ratio, which we have already explained is due to the similarity in the behavior of the variables used to calculate it. Even so, the ratio by itself could help us in future studies to know the tourist supply and demand in a province and give us an idea of the predominant type of tourist establishment in each area.